# Building A Database For Crime Reports
We've been tasked with building a database using Postgres that stores crime report information. This database should allow the data analysts read-only priviliges and the data scientists read-write priviliges.

## Database and Schema Creation

In [1]:
import psycopg2

# connect to database dq
conn = psycopg2.connect(dbname='dq', user='dq')
cur = conn.cursor()

# create database crime_db
conn.autocommit = True
cur.execute('''DROP DATABASE IF EXISTS crime_db;''')
cur.execute('''CREATE DATABASE crime_db;''')
conn.autocommit = False
conn.close()

# create schema crimes
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute('''CREATE SCHEMA crimes;''')

## Data Exploration

In [2]:
import csv

# place header and first row in global variables
col_headers = []
first_row = []
with open('boston.csv', 'r') as file:
    reader = csv.reader(file)
    reader_rows = []
    for row in reader:
        reader_rows.append(row)
    col_headers = reader_rows[0]
    first_row = reader_rows[1]
    
print('HEADERS: ' + str(col_headers))
print('\nFIRST ROW: ' + str(first_row))

HEADERS: ['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']

FIRST ROW: ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [3]:
# create function that returns a set of all column values from a given
# column index and csv file
def get_col_set(csv_filename, col_index):
    with open(csv_filename, 'r') as file:
        next(file)
        reader = csv.reader(file)
        distinct_col_values = set()
        for row in reader:
            distinct_col_values.add(row[col_index])
        return distinct_col_values

# print headers and corresponding count of distinct values in header column 
print('HEADERS: DISTINCT VALUES\n')
for col_index in range(len(col_headers)):
    col_distinct_total = len(get_col_set('boston.csv', col_index))
    print(col_headers[col_index] + ': ' + str(col_distinct_total))

HEADERS: DISTINCT VALUES

incident_number: 298329
offense_code: 219
description: 239
date: 1177
day_of_the_week: 7
lat: 18177
long: 18177


In [4]:
# get the longest length of offense_code and description for optimal
# datatype initialization
max_offense_code = 0
for offense_code in get_col_set('boston.csv', 1):
    offense_code_length = len(offense_code)
    if offense_code_length > max_offense_code:
        max_offense_code = offense_code_length
max_description = 0
for description in get_col_set('boston.csv', 2):
    description_length = len(description)
    if description_length > max_description:
        max_description = description_length

print('HEADERS: MAX LENGTH\n')
print(col_headers[1] + ': ' + str(max_offense_code))
print(col_headers[2] + ': ' + str(max_description))

HEADERS: MAX LENGTH

offense_code: 4
description: 58


## Table Creation
Let's create the crimes table using optimal datatypes for efficient storage. We'll print the column headers and the first row again to help with choosing the datatypes.

In [5]:
print('HEADERS: ' + str(col_headers))
print('\nFIRST ROW: ' + str(first_row))

HEADERS: ['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']

FIRST ROW: ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [6]:
# create enum datatype storing days of the week
cur.execute('DROP TYPE IF EXISTS days_of_the_week')
cur.execute('''
    CREATE TYPE days_of_the_week
    AS ENUM ('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday');
''')

# create crimes table
cur.execute('DROP TABLE IF EXISTS crimes.boston_crimes')
cur.execute('''
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code NUMERIC(4, 0),
        description VARCHAR(70),
        date DATE,
        day_of_the_week DAYS_OF_THE_WEEK,
        latitude NUMERIC(10, 8),
        longitude NUMERIC(11, 8)
    );
''')

In [7]:
# load the data from file boston.csv to crimes table
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

## Group Creation

In [8]:
# revoke public priviliges
cur.execute('REVOKE ALL ON SCHEMA public FROM public;')
cur.execute('REVOKE ALL ON DATABASE crime_db FROM public;')

In [9]:
# create groups readonly and readwrite
cur.execute('CREATE GROUP readonly NOLOGIN;')
cur.execute('CREATE GROUP readwrite NOLOGIN;')

# grant connections
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readonly;')
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readwrite;')

# grant usages
cur.execute('GRANT USAGE ON SCHEMA crimes TO readonly;')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readwrite;')

# grant priviliges
cur.execute('GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;')
cur.execute('GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;')

In [10]:
# create users
cur.execute('CREATE USER data_analyst WITH PASSWORD \'secret1\';')
cur.execute('CREATE USER data_scientist WITH PASSWORD \'secret2\';')

# assign users to groups
cur.execute('GRANT readonly TO data_analyst;')
cur.execute('GRANT readwrite TO data_scientist;')

In [11]:
# check readonly priviliges
cur.execute('''
    SELECT grantee, privilege_type
        FROM information_schema.table_privileges
        WHERE grantee = 'readonly';
''')
cur.fetchall()

[('readonly', 'SELECT')]

In [12]:
# check readwrite priviliges
cur.execute('''
    SELECT grantee, privilege_type
        FROM information_schema.table_privileges
        WHERE grantee = 'readwrite';
''')
cur.fetchall()

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]